In [1]:
%load_ext autoreload
%load_ext autotime

time: 313 µs (started: 2023-03-07 22:07:41 -08:00)


In [2]:
%run init.ipynb

/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': None, 'dashboard_address': ':8787', 'security': None}}
time: 3.87 s (started: 2023-03-07 22:07:41 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 959 ms (started: 2023-03-07 22:07:45 -08:00)


In [4]:
dask.__version__

'2021.08.1'

time: 9.56 ms (started: 2023-03-07 22:07:45 -08:00)


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.129.185:8787/status,
Dashboard: http://192.168.129.185:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.129.185:42525,Workers: 0
Dashboard: http://192.168.129.185:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 44.6 ms (started: 2023-03-07 22:07:46 -08:00)


In [6]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 678 µs (started: 2023-03-07 22:07:46 -08:00)


In [7]:
DELTA = 0.0001

time: 3.46 ms (started: 2023-03-07 22:07:46 -08:00)


In [8]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/mummi-wemul/nodes-32-ppn-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=914,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 703 ms (started: 2023-03-07 22:07:52 -08:00)


In [9]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,68024,0,22598272,2,0,3.027607,3.027607,MPI_Comm_size,0,...,wemul,,UNKNOWN,0,0,0.0,0.000000,30276069,0,0
1,2,68024,0,22598272,2,0,3.027650,3.027650,MPI_Comm_rank,0,...,wemul,,UNKNOWN,0,0,0.0,0.000000,30276498,0,0
2,3,68024,0,22598272,0,3,6.323863,6.324036,open,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000173,63239492,3341432784288721016,-7155284101858470952
3,4,68024,0,22598272,0,3,6.324068,6.324075,open,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000007,63240712,3341432784288721016,-7155284101858470952
4,5,68024,0,22598272,0,3,6.324091,6.324104,ftruncate,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000013,63240972,3341432784288721016,-7155284101858470952


time: 16.6 s (started: 2023-03-07 22:07:53 -08:00)


In [10]:
ddf.index.count().compute()

897296255

time: 20.4 s (started: 2023-03-07 22:08:10 -08:00)


In [11]:
ddf['tend'].max().compute()

3722.03271484375

time: 1.68 s (started: 2023-03-07 22:08:30 -08:00)


In [12]:
ddf['file_id'].unique().count().compute()

2243274

time: 1.61 s (started: 2023-03-07 22:08:32 -08:00)


In [13]:
ddf['proc_id'].unique().count().compute()

14

time: 1.32 s (started: 2023-03-07 22:08:34 -08:00)


In [15]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmid_min, tmid_max

(0, 37220326429)

time: 3.78 ms (started: 2023-03-07 22:09:11 -08:00)


In [16]:
divmod(tmid_max, 1e7)

(3722.0, 326429.0)

time: 2.51 ms (started: 2023-03-07 22:09:11 -08:00)


In [17]:
tmid_max_s = tmid_max / 1e7
tmid_max_s

3722.0326429

time: 2.99 ms (started: 2023-03-07 22:09:11 -08:00)


In [18]:
tmid_max_s / (60 * 60 * 24 * 30)

0.0014359693838348767

time: 2.38 ms (started: 2023-03-07 22:09:12 -08:00)


In [19]:
60 * 60 * 24 * 30

2592000

time: 2.19 ms (started: 2023-03-07 22:09:13 -08:00)


In [20]:
cm_tmid_max_s = 6683055996 / 1e7
cm_tmid_max_s

668.3055996

time: 2.02 ms (started: 2023-03-07 22:09:13 -08:00)


In [21]:
trange = np.arange(tmid_min, tmid_max, 1e7)
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 11.1 ms (started: 2023-03-07 22:09:13 -08:00)


In [22]:
max_io_time = ddf.groupby(['proc_id']).sum()['duration'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 17.1 ms (started: 2023-03-07 22:09:13 -08:00)


In [23]:
VIEW_TYPES = ['trange', 'file_id', 'proc_id']

time: 554 µs (started: 2023-03-07 22:09:15 -08:00)


In [24]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(ddf: dd.DataFrame, delta: float, max_io_time: float, metric='duration'):
    def set_delta(df: pd.DataFrame):
        df['duration_csp'] = df['duration_sum'].cumsum() / max_io_time
        df['duration_delta'] = df['duration_csp'].diff().fillna(df['duration_csp'])
        df['duration_score'] = np.digitize(df['duration_delta'], bins=DELTA_BINS, right=True)
        df['duration_cut'] = np.choose(df['duration_score'] - 1, choices=DELTA_BINS, mode='clip')
        return df
    ddf = ddf.map_partitions(set_delta)
    return ddf[ddf['duration_delta'] > delta]

time: 1.88 ms (started: 2023-03-07 22:09:17 -08:00)


In [25]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[ddf['cat'] == 0] \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first']
#         'func_id': [unique_flatten()]
    }) \
    .reset_index() \
    .persist()

main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]

main_view


,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name_first,proc_name_first
npartitions=1,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float32,int64,int64,int64,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...


time: 432 ms (started: 2023-03-07 22:09:31 -08:00)


In [26]:
wait(main_view)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('reset_index-27d10ff8669acd486f084a6e7ddae1a2', 0)>}, not_done=set())

time: 50.9 s (started: 2023-03-07 22:09:32 -08:00)


In [27]:
main_view.head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name_first,proc_name_first
0,1,113104469769545699,-7155286342531755048,3,0,0.037957,3690,0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104
1,1,113104469769545699,-7155286342526053416,3,0,0.000037,28,0,0,0,UNKNOWN,wemul#lassen736#153477#35184394687104
2,1,113104469769545699,-7155286341227260968,3,0,0.003808,508,0,0,0,UNKNOWN,wemul#lassen736#107760#35184394687104
3,1,113104469769545699,-7155284101313735720,3,0,0.061947,4356,0,0,0,UNKNOWN,wemul#lassen83#76336#35184394687104
4,1,113104469769545699,-7155284099504286760,3,0,2.344551,24684,0,0,0,UNKNOWN,wemul#lassen83#169481#35184394687104


time: 100 ms (started: 2023-03-07 22:10:23 -08:00)


In [28]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
#main_view['metadata_count'] = main_view['metadata_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)

final_view = main_view \
    .groupby(['trange', 'file_id', 'proc_id']) \
    .agg({
        'duration_sum': sum,
        'index_count': sum,
        'size_sum': sum,
        'read_count': sum,
        'write_count': sum,
        'metadata_count': sum,
        'read_size': sum,
        'write_size': sum,
        'metadata_size': sum,
        'read_time': sum,
        'write_time': sum,
        'metadata_time': sum,
        'sequential_count': sum,
        'random_count': sum,
        'sequential_size': sum,
        'random_size': sum,
        'sequential_time': sum,
        'random_time': sum,
        'data_count': sum,
        'data_size': sum,
        'data_time': sum,
#         'workload_type': sum,
        'size_min': min,
        'size_max': max,
        'file_name_first': 'first',
        'proc_name_first': 'first'
    }) \
    .reset_index() \
    .rename(columns={
        'file_name_first': 'file_name', 
        'proc_name_first': 'proc_name'
    })
mask_hostname = int('0000000000000000111111111111111100000000000000000000000000000000', 2)

final_view['hostname_id'] = final_view['proc_id'] & mask_hostname
final_view = final_view.persist()


time: 373 ms (started: 2023-03-07 22:10:41 -08:00)


In [29]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('assign-717fcfd3e0e6129a11d5395ae4043ce9', 0)>}, not_done=set())

time: 53.9 s (started: 2023-03-07 22:10:43 -08:00)


In [30]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name,hostname_id
0,1,113104469769545699,-7155286342531755048,0.037957,3690,0,0,0,3690,0,...,0.037957,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104,89038967013376
1,1,113104469769545699,-7155286342526053416,0.000037,28,0,0,0,28,0,...,0.000037,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#153477#35184394687104,89038967013376
2,1,113104469769545699,-7155286341227260968,0.003808,508,0,0,0,508,0,...,0.003808,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#107760#35184394687104,89038967013376
3,1,113104469769545699,-7155284101313735720,0.061947,4356,0,0,0,4356,0,...,0.061947,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen83#76336#35184394687104,91280939941888
4,1,113104469769545699,-7155284099504286760,2.344551,24684,0,0,0,24684,0,...,2.344551,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen83#169481#35184394687104,91280939941888


time: 106 ms (started: 2023-03-07 22:11:37 -08:00)


In [30]:
del main_view

time: 648 µs (started: 2023-03-04 17:22:10 -08:00)


In [31]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name,hostname_id
0,1,113104469769545699,-7155286342531755048,0.037957,3690,0,0,0,3690,0,...,0.037957,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104,89038967013376
1,1,113104469769545699,-7155286342526053416,0.000037,28,0,0,0,28,0,...,0.000037,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#153477#35184394687104,89038967013376
2,1,113104469769545699,-7155286341227260968,0.003808,508,0,0,0,508,0,...,0.003808,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen736#107760#35184394687104,89038967013376
3,1,113104469769545699,-7155284101313735720,0.061947,4356,0,0,0,4356,0,...,0.061947,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen83#76336#35184394687104,91280939941888
4,1,113104469769545699,-7155284099504286760,2.344551,24684,0,0,0,24684,0,...,2.344551,0,0,0,0.0,0,0,UNKNOWN,wemul#lassen83#169481#35184394687104,91280939941888


time: 54.5 ms (started: 2023-03-07 22:12:14 -08:00)


In [32]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(view_type: str, ddf: dd.DataFrame, delta: float, metric: str, max_io_time: dd.core.Scalar):
    metric_col, csp_col, delta_col, score_col, cut_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
        f"{metric}_score",
        f"{metric}_cut"
    )

    def set_delta(df: pd.DataFrame):
        df[csp_col] = df[metric_col].cumsum() / max_io_time
        df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
        df[score_col] = np.digitize(df[delta_col], bins=DELTA_BINS, right=True)
        df[cut_col] = np.choose(df[score_col] - 1, choices=DELTA_BINS, mode='clip')
        return df

    index = pd.MultiIndex(levels=[[1]], codes=[[]], names=[view_type])
    meta = dd.utils.make_meta(([
        (metric_col, np.float64),
        (csp_col, np.float64),
        (delta_col, np.float64),
        (score_col, np.int8),
        (cut_col, np.float16),
    ]), index=index)

#     ddf = ddf.map_partitions(set_delta, meta=meta)
    ddf = ddf.map_partitions(set_delta)

    return ddf[ddf[delta_col] > delta]


time: 3.06 ms (started: 2023-03-07 22:12:18 -08:00)


In [33]:
max_io_time = final_view.groupby(['proc_id']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 22.3 ms (started: 2023-03-07 22:12:30 -08:00)


In [38]:
import itertools as it

def _set_metric_percentages(ddf: dd.DataFrame, max_io_time: dd.core.Scalar, metric: str):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    ddf[pero_col] = ddf[metric_col] / max_io_time
    ddf[perr_col] = ddf[metric_col] / ddf[metric_col].sum()
    return ddf

def _compute_expanded_view(
    parent_view: dd.DataFrame,
    view_type: str,
    cols: tuple,
    metric: str,
    delta: float,
    max_io_time: dd.core.Scalar
):
    # Get column names
    metric_col, score_col, cut_col = cols
    # Check view type
    if view_type is not 'proc_id':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_id']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Filter view
    filtered_view = filter_delta(
        view_type=view_type,
        ddf=group_view,
        delta=delta,
        metric=metric,
        max_io_time=max_io_time
    )
        
    # Get score view
    score_view = filtered_view.reset_index()[[view_type, score_col, cut_col]]
    # Find filtered records and set duration scores
#     expanded_view = parent_view \
#         .query(f"{view_type}.isin(@indices)", local_dict={'indices': filtered_view.index.unique()}) \
#         .drop(columns=[score_col, cut_col], errors='ignore') \
#         .merge(score_view, on=[view_type])
    expanded_view = parent_view \
        .query(f"{view_type} in @indices", local_dict={'indices': filtered_view.index.unique()}) \
        .drop(columns=[score_col, cut_col], errors='ignore') \
        .merge(score_view, on=[view_type])
    
    # Set metric percentages
    expanded_view = _set_metric_percentages(ddf=expanded_view, max_io_time=max_io_time, metric=metric)
    # Return expanded view
    return expanded_view

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type]['expanded_view'] if parent_type in views else final_view
    # Create colum names
    metric_col, score_col, cut_col = f"{metric}_sum", f"{metric}_score", f"{metric}_cut"
    # Compute expanded view
    expanded_view = _compute_expanded_view(
        parent_view=parent_view,
        view_type=view_type,
        cols=(metric_col, score_col, cut_col),
        metric=metric,
        delta=0.0001,
        max_io_time=max_io_time
    )
    # Return views
    views[view_permutation] = {}
    views[view_permutation]['expanded_view'] = expanded_view
    views[view_permutation]['bottleneck_view'] = expanded_view[expanded_view[cut_col] >= 0.5]

len(views)

15

time: 4.44 s (started: 2023-03-07 22:16:57 -08:00)


In [39]:
for view_key, view_dict in views.items():
    print(view_key, 'expanded_view', len(view_dict['expanded_view']), 'bottleneck_view', len(view_dict['bottleneck_view']))

('trange',) expanded_view 9268841 bottleneck_view 0
('file_id',) expanded_view 19744 bottleneck_view 0
('proc_id',) expanded_view 9290293 bottleneck_view 8642201
('trange', 'file_id') expanded_view 19616 bottleneck_view 0
('trange', 'proc_id') expanded_view 9268841 bottleneck_view 8639769
('file_id', 'trange') expanded_view 14389 bottleneck_view 0
('file_id', 'proc_id') expanded_view 19724 bottleneck_view 0
('proc_id', 'trange') expanded_view 9268841 bottleneck_view 0
('proc_id', 'file_id') expanded_view 19744 bottleneck_view 0
('trange', 'file_id', 'proc_id') expanded_view 19601 bottleneck_view 0
('trange', 'proc_id', 'file_id') expanded_view 19616 bottleneck_view 0
('file_id', 'trange', 'proc_id') expanded_view 14377 bottleneck_view 0
('file_id', 'proc_id', 'trange') expanded_view 14377 bottleneck_view 0
('proc_id', 'trange', 'file_id') expanded_view 19616 bottleneck_view 0
('proc_id', 'file_id', 'trange') expanded_view 14389 bottleneck_view 0
time: 4min 51s (started: 2023-03-07 22:1

In [40]:
views[('trange',)]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,data_time,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
0,4,113104469769545699,-7155286342531755048,0.222302,31007,0,0,0,31007,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,1.073723e-07,3.403557e-08
1,4,113104469769545699,-7155286342526053416,0.000284,188,0,0,0,188,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#153477#35184394687104,89038967013376,1,0.0,1.372207e-10,4.349712e-11
2,4,113104469769545699,-7155286342512749608,0.586636,257518,0,0,0,257518,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#22610#35184394687104,89038967013376,1,0.0,2.833460e-07,8.981688e-08
3,4,113104469769545699,-7155286342497545256,52.501892,257340,0,0,0,257340,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#153912#35184394687104,89038967013376,1,0.0,2.535848e-05,8.038299e-06
4,4,113104469769545699,-7155286341227260968,0.023181,3770,0,0,0,3770,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#107760#35184394687104,89038967013376,1,0.0,1.119645e-08,3.549125e-09


time: 6.07 s (started: 2023-03-07 22:21:53 -08:00)


In [41]:
views[('proc_id',)]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,data_time,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
0,1,113104469769545699,-7155286342531755048,0.037957,3690,0,0,0,3690,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,1.833314e-08,5.804185e-09
1,1,3341432780469747200,-7155286342531755048,0.000019,4,0,0,0,4,0,...,0.0,0,0,/dev/shm/job1792606209-26990-OMPI_COLL_IBM-0-c...,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,9.273625e-12,2.935987e-12
2,1,3341432780510128914,-7155286342531755048,0.000017,4,0,0,0,4,0,...,0.0,0,0,/dev/shm/job1792606209-26990-OMPI_COLL_IBM-0-c...,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,8.162722e-12,2.584280e-12
3,1,3341432780598284696,-7155286342531755048,0.000022,4,0,0,0,4,0,...,0.0,0,0,/dev/shm/job1792606209-26990-OMPI_COLL_IBM-0-c...,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,1.062603e-11,3.364151e-12
4,1,3341432780659229742,-7155286342531755048,0.000016,4,0,0,0,4,0,...,0.0,0,0,/dev/shm/job1792606209-26990-OMPI_COLL_IBM-0-c...,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,7.583121e-12,2.400780e-12


time: 6.8 s (started: 2023-03-07 22:21:59 -08:00)


In [43]:
views[('file_id',)]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,data_time,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
0,1,113104469769545699,-7155286342531755048,0.037957,3690,0,0,0,3690,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#22320#35184394687104,89038967013376,1,0.0,1.833314e-08,1.030719e-08
1,1,113104469769545699,-7155286342526053416,0.000037,28,0,0,0,28,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#153477#35184394687104,89038967013376,1,0.0,1.777445e-11,9.993091e-12
2,1,113104469769545699,-7155286341227260968,0.003808,508,0,0,0,508,0,...,0.0,0,0,UNKNOWN,wemul#lassen736#107760#35184394687104,89038967013376,1,0.0,1.839221e-09,1.034041e-09
3,1,113104469769545699,-7155284101313735720,0.061947,4356,0,0,0,4356,0,...,0.0,0,0,UNKNOWN,wemul#lassen83#76336#35184394687104,91280939941888,1,0.0,2.992039e-08,1.682174e-08
4,1,113104469769545699,-7155284099504286760,2.344551,24684,0,0,0,24684,0,...,0.0,0,0,UNKNOWN,wemul#lassen83#169481#35184394687104,91280939941888,1,0.0,1.132421e-06,6.366660e-07


time: 3.83 s (started: 2023-03-07 22:22:16 -08:00)


In [44]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_id=('file_name', 'proc_name', 'trange'),
    proc_id=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_id='file_name',
    proc_id='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    ll_view: pd.DataFrame,
    ml_view: pd.DataFrame,
    hl_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = ll_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = hl_view.loc[hl_id]
        ml_row = ml_view.loc[(hl_id, ml_id)]
        ll_row = ll_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view_dict in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view_dict=view_dict,
            view_types=view_types,
        )
    # Generate bottlenecks
#     bottlenecks = _process_bottleneck_views(bottleneck_views=bottleneck_views)
    # Return bottleneck views
    return bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, pd.DataFrame]):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        bottlenecks_delayed.append(_process_bottleneck_view(
            view_key=view_key,
            ll_view=view_dict['low_level_view'],
            ml_view=view_dict['mid_level_view'],
            hl_view=view_dict['high_level_view']
        ))
    # Compute all bottlenecks
    futures = compute(*bottlenecks_delayed, sync=False)
    results = get_client().gather(list(futures))
    for view_key, result in results:
        bottlenecks[view_key] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
        view_key: tuple,
        view_dict: Dict[str, dd.DataFrame],
        view_types: list
    ):
        # Get view type
        view_type = view_key[-1]
        bottleneck_type = BOTTLENECK_TYPE[view_type]

        # Get parent view
        bottleneck_view = view_dict['bottleneck_view']

        # Create lower level view
        low_level_view = bottleneck_view \
            .groupby(list(BOTTLENECK_ORDER[view_type])) \
            .first() \
            .drop(columns=['acc_pat', 'io_cat'], errors='ignore')

        # Non-proc agg columns
        non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
        proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

        if bottleneck_type is not 'proc_name':

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'proc_name']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(proc_agg_dict)

        else:

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'trange']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(non_proc_agg_dict)

        return dict(
            low_level_view=low_level_view.persist(),
            mid_level_view=mid_level_view.persist(),
            high_level_view=high_level_view.persist()
        )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict['duration_cut'] = 'first'
    agg_dict['duration_score'] = 'first'
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 7.01 ms (started: 2023-03-07 22:22:22 -08:00)


In [45]:
bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottleneck_views)

15

time: 3.68 s (started: 2023-03-07 22:22:24 -08:00)


In [46]:
for view_key, bottleneck_view_dict in bottleneck_views.items():
    print(view_key, 'hl_view', len(bottleneck_view_dict['high_level_view']), 'ml_view', len(bottleneck_view_dict['mid_level_view']), 'll_view', len(bottleneck_view_dict['low_level_view']))

('trange',) hl_view 0 ml_view 0 ll_view 0
('file_id',) hl_view 0 ml_view 0 ll_view 0
('proc_id',) hl_view 2 ml_view 7198 ll_view 8642201
('trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'proc_id') hl_view 2 ml_view 7143 ll_view 8639769
('file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('file_id', 'proc_id') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'file_id', 'proc_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('file_id', 'trange', 'proc_id') hl_view 0 ml_view 0 ll_view 0
('file_id', 'proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
time: 1min 56s (started: 2023-03-07 22:22:29 -08:00)


In [49]:
max_io_time.compute()

2070387.75

time: 229 ms (started: 2023-03-07 22:25:45 -08:00)


In [47]:
bottleneck_views[('trange', 'proc_id')]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,data_count,data_size,data_time,size_min,size_max,hostname_id,duration_pero,duration_perr,duration_score,duration_cut
proc_name,,,,,,,,,,,,,,,,,,,,,
wemul#lassen83#132846#35184394687104,2062567.750,346699813,66332177717072,126173557,63601738,156924518,65971207924648,360969792424,0,13083.675579,...,189775295,66332177717072,644592.134302,0,1048576,3.993894e+20,0.996223,0.315789,6,0.75
wemul#lassen83#87704#35184394687104,1949690.375,346826752,66358261448704,126198784,63651840,156976128,65971208126464,387053322240,0,12768.443329,...,189850624,66358261448704,634994.015091,0,1048576,3.892568e+20,0.941703,0.298507,6,0.75


time: 29.3 ms (started: 2023-03-07 22:24:56 -08:00)


In [48]:
bottleneck_views[('trange', 'proc_id')]['mid_level_view'].compute()

duration_sum  index_count  \
proc_name                            trange                              
wemul#lassen83#132846#35184394687104 6           5.150005       142102   
                                     7          86.249802      2442602   
                                     8           0.546534        15264   
                                     12         24.499636       685856   
                                     13         71.132805      1948807   
...                                                   ...          ...   
wemul#lassen83#87704#35184394687104  3485       33.662056         2524   
                                     3486       57.529156         1786   
                                     3487       34.090763         2270   
                                     3488       22.073017          772   
                                     3489        2.069042          243   

                                               size_sum  read_count  \
proc_name                            trange                           
wemul#lassen83#132846#35184394687104 6                0           0   
                                     7                0           0   
                                     8                0           0   
                                     12               0           0   
                                     13      1689281648        4238   
...                                                 ...         ...   
wemul#lassen83#87704#35184394687104  3485     503324416         496   
                                     3486     378541552         351   
                                     3487     467671936         440   
                                     3488     158337184         138   
                                     3489      41943680          40   

                                             write_count  metadata_count  \
proc_name                            trange                                
wemul#lassen83#132846#35184394687104 6                 0          142102   
                                     7                 0         2442602   
                                     8                 0           15264   
                                     12                0          685856   
                                     13             1607         1942962   
...                                                  ...             ...   
wemul#lassen83#87704#35184394687104  3485            976            1052   
                                     3486            712             723   
                                     3487            886             944   
                                     3488            289             345   
                                     3489             80             123   

                                              read_size  write_size  \
proc_name                            trange                           
wemul#lassen83#132846#35184394687104 6                0           0   
                                     7                0           0   
                                     8                0           0   
                                     12               0           0   
                                     13      1689268792       12856   
...                                                 ...         ...   
wemul#lassen83#87704#35184394687104  3485          3968   503320448   
                                     3486          2808   378538744   
                                     3487          3520   467668416   
                                     3488          1104   158336080   
                                     3489           320    41943360   

                                             metadata_size  read_time  ...  \
proc_name                            trange                            ...   
wemul#lassen83#132846#35184394687104 6                   0   0.000000  ...   
           

time: 47.3 ms (started: 2023-03-07 22:25:27 -08:00)


In [52]:
bottleneck_views[('trange', 'proc_id')]['low_level_view'].compute()

file_id  \
proc_name                            trange file_name                                                                 
wemul#lassen83#132846#35184394687104 6      /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...  3341432782915608019   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...  3341432782742116205   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...  3341432782221255945   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...  3341432784104156847   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...  3341432783301287596   
...                                                                                                             ...   
wemul#lassen83#87704#35184394687104  3489   /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664...  5347682699610306931   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664...  5347682701186177475   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664...  5347682700713831203   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664...  5347682702556746497   
                                            UNKNOWN                                              113104469769545699   

                                                                                                            proc_id  \
proc_name                            trange file_name                                                                 
wemul#lassen83#132846#35184394687104 6      /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co... -7155284101905198120   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co... -7155284101905198120   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co... -7155284101905198120   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co... -7155284101905198120   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co... -7155284101905198120   
...                                                                                                             ...   
wemul#lassen83#87704#35184394687104  3489   /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664... -7155284100568722472   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664... -7155284100568722472   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664... -7155284100568722472   
                                            /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664... -7155284100568722472   
                                            UNKNOWN                                            -7155284100568722472   

                                                                                                duration_sum  \
proc_name                            trange file_name                                                          
wemul#lassen83#132846#35184394687104 6      /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...      0.179631   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...      0.171780   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...      0.193542   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...      0.186994   
                                            /dev/shm/job1946235062-26990-OMPI_COLL_IBM-0-co...      0.179453   
...                                                                                                      ...   
wemul#lassen83#87704#35184394687104  3489   /p/gpfs1/iopp/temp/wemul_mummi_emulation.324664...      0.039417   
    

time: 16.1 s (started: 2023-03-07 22:27:36 -08:00)
